<a href="https://colab.research.google.com/github/giorgiocerruti/llama32-training-demo/blob/main/train_llama_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [1]:
%%capture
!pip install unsloth

!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Download the model to train

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = False

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Setup the LoRA parameters

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = False,
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.12.12 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Example before training

In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1"
)
FastLanguageModel.for_inference(model)

message = [
    {"role": "user", "content": "flux template a girl rifing a bike ina a sunset realistic"},
]

inputs = tokenizer.apply_chat_template(
    message,
    tokenizer = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Here's a Flux template to create a realistic sunset scene with a girl riding a bike:

**Title:** Sunset Ride

**Description:** A serene and peaceful sunset scene of a girl riding a bike on a winding road with a stunning view of the setting sun.

**Template:**

```glitch
# Setup
export default {
  version: '1',
  experiment: 'flutter',
  resolution: 1080,
  aspect: '21:9',
  sampling: true,
  sampler: 'h264',
  hardware: false
};

// Image
image: {
  url: 'https://example.com/image.jpg',


# Import traning dataset

In [ ]:
import pandas as pd
df = pd.read_csv("flux-training-dataset.csv")

df.head()

# Remove blank lines

In [6]:
df = df.dropna()

# Set dataset in the llama way template

In [7]:
from datasets import Dataset

df["conversations"] = df.apply(
    lambda x: [
        {"content": x["User"], "role": "user"},
        {"content": x["Prompt"], "role": "assistant"},
    ], axis=1
)
# drop the colums that we don't use fr training
dataset = Dataset.from_pandas(df.drop(columns=["User", "Prompt"]))

<ipython-input-7-3cc5aa8b5b65>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["conversations"] = df.apply(


# Print test

In [8]:
dataset["conversations"][0]

[{'content': '"flux prompt for a Viking warrior with a black iron amulet"',
  'role': 'user'},
 {'content': '"Norse amulet made from black iron with a carbon fiber necklace, matte painting, Unreal Engine, --ar 16:9"',
  'role': 'assistant'}]

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "conversations",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none"
    ),

)

Map (num_proc=2):   0%|          | 0/287 [00:00<?, ? examples/s]

# Traing setup

In [10]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/287 [00:00<?, ? examples/s]

# Traning

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 287 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,4.897000
2,4.927300
3,4.874600
4,4.521300
5,4.550000
6,4.529300
7,4.283400
8,3.818800
9,3.681700
10,3.354300


# Inferenza

In [12]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "flux template a girl rifing a bike ina a sunset realistic"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

"a young girl on a bike, her dark hair is blown back in the wind, a fierce look is set on her face and her arm is raised in the air, her bike wheels spin wildly as the girl hurtles down a steep hill, her eyes are closed and her smile is frozen in time. a beautiful, realistic sunset looms large behind the girl, a warm glow of orange, red and yellow that stretches across the entire scene, giving the image a sense of dynamic energy."<|eot_id|>


# Compile llama.cpp
This is a workarond to make the
`Esporto GGUF per Ollama`
working

In [23]:
!(cd llama.cpp; cmake -B build;cmake --build build --config Release)
!cp llama.cpp/build/bin/* /content/llama.cpp

-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend
-- x86 detected
-- Adding CPU backend variant ggml-cpu: -march=native 
-- CUDA Toolkit found
-- Using CUDA architectures: native
-- CUDA host compiler is GNU 11.4.0

-- Including CUDA backend
-- Configuring done (0.4s)
-- Generating done (0.4s)
-- Build files have been written to: /content/llama.cpp/build
[  3%] Built target ggml-base
[ 29%] Built target ggml-cuda
[ 33%] Built target ggml-cpu
[ 33%] Built target ggml
[ 40%] Built target llama
[ 40%] Built target build_info
[ 44%] Built target common
[ 45%] Built target test-tokenizer-0
[ 46%] Built target test-sampling
[ 47%] Built target test-grammar-parser
[ 48%] Built target test-grammar-integration
[ 49%] Built target test-llama-grammar
[ 50%] Built target test-json-schema-to-grammar
[ 50%] Built target test-tokenizer-1-bpe
[ 51%] Built target test-token

# Export GGUF for make it runnable for Ollama

In [24]:
#model.save_pretrained_merged(, tokenizer, save_method = "merged_16bit",)
model.save_pretrained_gguf("/content/drive/MyDrive/model", tokenizer, quantization_method = "f16")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.39 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:00<00:00, 32.41it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving /content/drive/MyDrive/model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving /content/drive/MyDrive/model/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at /content/drive/MyDrive/model into f16 GGUF format.
The output location will be /content/drive/MyDrive/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32

# Zip the wxpoted model

In [25]:
!zip -r /content/llama-32-flux-trained.zip /content/drive/MyDrive

  adding: content/drive/MyDrive/ (stored 0%)
  adding: content/drive/MyDrive/model/ (stored 0%)
  adding: content/drive/MyDrive/model/unsloth.F16.gguf (deflated 10%)
  adding: content/drive/MyDrive/model/tokenizer.json (deflated 85%)
  adding: content/drive/MyDrive/model/generation_config.json (deflated 38%)
  adding: content/drive/MyDrive/model/Modelfile (deflated 65%)
  adding: content/drive/MyDrive/model/pytorch_model.bin.index.json (deflated 96%)
  adding: content/drive/MyDrive/model/tokenizer_config.json (deflated 94%)
  adding: content/drive/MyDrive/model/pytorch_model-00002-of-00002.bin (deflated 8%)
  adding: content/drive/MyDrive/model/pytorch_model-00001-of-00002.bin (deflated 10%)
  adding: content/drive/MyDrive/model/special_tokens_map.json (deflated 71%)
  adding: content/drive/MyDrive/model/config.json (deflated 52%)


In [26]:
from google.colab import files
files.download('/content/llama-32-flux-trained.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')